# Distributed Computing with dask

In this portion of the course, we'll explore distributed computing with a Python library called `dask`. 

`dask` is a library designed to help facilitate (a) manipulation of very large datasets, and (b) distribution of computation across lots of cores or physical computers. It is very similar to Apache Spark in the functionality it provides, but it is tightly integrated into `numpy` and `pandas`, making it *much* easier to learn than spark for users of those libraries. 

## What can dask do for me?

To get a sense of why `dask` is so nice, let's begin with a demonstration. Suppose I have a pretty big dataset (say, records of all drug shipments in the United States from 2006 to 2012). This data is too large for me to load into ram on my laptop directly, so if I were to work with it on my own, I'd do so by [chunking the data by hand](big_data_strategies.ipynb). But using `dask`, I can do the following: 

In [2]:
# Standard setups
import pandas as pd
import os
os.chdir('/users/nick/github/practicaldatascience/example_data/dask_data')

In [4]:
# Here we start up a dask cluster. 
# This creates a set of "workers". 
# In this case, these workers are all on 
# my computer, but if we wanted to connect
# to a cluster, we'd just past an IP address
# to `Client()`. 

from dask.distributed import Client
client = Client(n_workers=7)
client

Client Scheduler: tcp://127.0.0.1:62038 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 21 Memory: 34.36 GB


In [6]:
# ARCOS Data on all drug shipments in US from 2006 to 2012

# Note that only the compressed version of this data
# is available in the practical data science repo, 
# and while pandas can read compressed files like this, 
# `dask` can not. So if you want to do this at home, 
# you have to decompress the data. 

import dask.dataframe as dd

df = dd.read_csv('arcos_all_washpost.tsv', delimiter='\t', 
                dtype={'ACTION_INDICATOR': 'object',
                       'ORDER_FORM_NO': 'object',
                       'REPORTER_ADDRESS2': 'object',
                       'REPORTER_ADDL_CO_INFO': 'object',
                       'NDC_NO': 'object',
                       'UNIT': 'object'})

In [7]:
df['date'] = dd.to_datetime(df.TRANSACTION_DATE, format="%m%d%Y")
df['year'] = df.date.dt.year
df['morphine_equivalent_g'] = (df['CALC_BASE_WT_IN_GM']) * df['MME_Conversion_Factor']
df = df[['year', 'morphine_equivalent_g', 'BUYER_STATE', 'BUYER_COUNTY']]
collapsed = df.groupby(['year', 'BUYER_STATE', 'BUYER_COUNTY']).morphine_equivalent_g.sum()
collapsed

Dask Series Structure:
npartitions=1
    float64
        ...
Name: morphine_equivalent_g, dtype: float64
Dask Name: series-groupby-sum-agg, 18931 tasks

In [ ]:
%%time
final = collapsed.compute()
final.sample(10)

Voila! I get back my thinned out dataset with the new variable I wanted, all while never using more than about 2gb of RAM at any point. 

So let's discuss what just happened. 

1. First, when I ran `Client()`, `dask` started up a set of 4 new python processes on my computer (called "workers") it could call on for help. These are fully independent agents that *could* be running on different machines in a true cluster (though in this case, they're all on my laptop).
2. It then collected all the instructions I gave it for reading in the file, generating a new column, deleting extra columns, and grouping. **But it didn't actually execute any code**. That's why when I typed `collapsed`, what I got back was a dataframe with structure but no data -- `dask` had figured out roughly what the result of my commands would look like, but hadn't actually computed the result. 
3. Then, when I ran `collapsed.compute()`, `dask` came up with a set of assignments it could give to its workers to get me what I wanted and it started to actually compute the table I requested. In particular:
    - each worker loaded a chunk of the data, did the observation-level manipulations I wanted, and dropped extra columns. 
    - then to execute the groupby, it then collapsed the data in each chunk, passed those collapsed chunks to a common worker who then re-collapsed them further, then gathered them again under (potentially) another worker and collapsed further until all the data had been collapsed. 
    - Finally, after all the data had been collapsed, the data was passed back to me (the client session) as the variable `final`. 

## Um... why didn't you tell us about this before?

It sure seems like this is easier than [chunking by hand](big_data_strategies)!

For a few reasons. 

1. This example was carefully chosen. This type of distributed processing is very good at *certain* manipulations, but it can also be very fragile. Do the wrong things, and you can easily crash your computer. 
2. The biggest challenge with a tool like `dask` or `pyspark` is that you have to understand what the software is doing in order to be able to give it commands that won't overwhelm your system. And the best way to do that is to get experience chunking by hand. 

For example, one of the key features of distributing computing is that they don't run code as soon as you type it. Instead, they just keep track of what you want and wait until you run `.compute()` to actually execute your code. Why? Because if `dask` had run each command as I entered it and pass the result back to me, it would have crashed my computer. 

Recall that the whole reason we're using dask is precisely because we knew that loading the whole dataset at once would take up more memory than I have available. It's only because we only loaded chunks of the data, thinned out those chunks by dropping variables, and then collapsed by county-state-year that we ended up with a dataset that was small enough that it would fit in memory.

In other words, this only worked because I knew that the steps after the intial load would reduce the size of the dataset enough that when I ran `.compute()`, the result I got back would be small enough to fit in memory. Had I just run:

```python
df = dd.read_csv('arcos_all_washpost.tsv', delimiter='\t', 
                dtype={'ACTION_INDICATOR': 'object',
                       'ORDER_FORM_NO': 'object',
                       'REPORTER_ADDRESS2': 'object',
                       'REPORTER_ADDL_CO_INFO': 'object',
                       'NDC_NO': 'object',
                       'UNIT': 'object'})
df.compute()
```

`dask` would have tried to hand the full dataset back to me, and my computer would have crashed, just as it would had I tried to read the full dataset with `pandas`. 

So understanding the *intuition* of chunking is basically a pre-requisite to using these tools effectively. 

## Vocabulary

Now that we've seen this example, let's formally introduce some distributed computing vocabulary:

- **Delayed Execution** (also sometimes called "lazy evaluation"): The practice of not executing code as soon as you type it, but rather accumulating a set of requests, then executing them when instructed. This allows distributing computing systems to optimize how they move around data and get things done, and ensures that you don't end up with the system trying to shove a massive dataset into ram that's too small. 
- **Client**: The central process (here, Python process) where code is being entered.
- **Workers**: Other processes that are assigned work, and which eventually pass results back to the client. 
- **Scheduler**: The part of the system that manages the assignment of tasks to different workers.
- **map-reduce**: The name for the process of distributing sub-problems to workers (`map`), then processing them in a way that allows the results to be recombined (`reduce`). `map`s and `reduce`s are kind of the building blocks of distributed systems, though when working with `dask` you won't usually have to manage the assignment or recombination of tasks yourself (they're happening behind the scenes). 

## Let's Learn More dask!

OK, now that we have the basic idea of `dask` in hand, please watch the following (*very* well organized) talk on `dask`. Then come back here and we can talk some more!

[Matthew Rocklin on Dask](https://www.youtube.com/watch?v=RA_2qdipVng)